ACTUAL TEST ACCURACY: 0.63%

In [ ]:
def CNNvanilla_augmentation(input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)

    # add augmentation at the beginning
    preprocessing = tf.keras.Sequential([
        #tfkl.RandomBrightness(0.2, value_range=(0,1)),
        tfkl.RandomTranslation(0.2,0.2),
        layers.RandomFlip(),
        layers.RandomZoom(height_factor=(0.2, 0.3)),
        
    ], name='preprocessing')

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    preprocessing = preprocessing(input_layer)

    x = tfkl.Conv2D(filters=32, kernel_size=3, padding='same', name='conv0')(preprocessing)
    x = tfkl.ReLU(name='relu0')(x)
    x = tfkl.MaxPooling2D(name='mp0')(x)

    x = tfkl.Conv2D(filters=64, kernel_size=3, padding='same', name='conv1')(x)
    x = tfkl.ReLU(name='relu1')(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)

    x = tfkl.Conv2D(filters=128, kernel_size=3, padding='same', name='conv2')(x)
    x = tfkl.ReLU(name='relu2')(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)

    x = tfkl.Conv2D(filters=256, kernel_size=3, padding='same', name='conv3')(x)
    x = tfkl.ReLU(name='relu3')(x)
    x = tfkl.MaxPooling2D(name='mp3')(x)

    x = tfkl.Conv2D(filters=512, kernel_size=3, padding='same', name='conv4')(x)
    x = tfkl.ReLU(name='relu4')(x)

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)

    x  = tfkl.Dense(units=128, kernel_initializer=tf.initializers.HeUniform(seed=seed), activation='relu',name='Output1')(x)
    x =  layers.Dropout(0.2)(x)

    output_layer = tfkl.Dense(units=2, activation='softmax',name='Output2')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CNN')

    # Compile the model
    model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

    # Return the model
    return model

In [ ]:
tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

augmented_model = CNNvanilla_augmentation()
augmented_model.summary()
callbacks = [
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, restore_best_weights=True, mode='auto'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)
]

In [ ]:

aug_history = CNNvanilla_augmentation.fit(
    x = X_train,
    y = y_train,
    batch_size = 32,
    epochs = 100,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history